<a href="https://colab.research.google.com/github/Yuri-Cordeiro/FarmTechSolutions/blob/exportacao_dados_agricultura/CulturaAgricola.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Gestão Agrícola

Este notebook implementa um sistema de gestão agrícola para calcular áreas de plantio e manejo de insumos para diferentes culturas. Atualmente, ele suporta as culturas de **Soja, Café, Milho, Feijão** e **Cana-de-Açúcar**.


## Importações e Definições Auxiliares

Nesta seção, importamos as bibliotecas necessárias e definimos funções auxiliares para o sistema, como a limpeza do terminal e a entrada de valores numéricos.


In [1]:
import io
import sys
import csv

import pandas as pd
import requests

import rpy2.robjects as robjects
import rpy2.rinterface_lib.callbacks
from rpy2.robjects import r
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.packages import importr
from IPython.display import clear_output
from datetime import datetime

# Captura a saída do R
utils = importr('utils')


def input_numerico(texto_input) -> int:
    """
    Solicita um input numérico do usuário e valida se é um número inteiro.

    :param texto_input: Texto a ser exibido para o usuário
    :return: Valor numérico inteiro
    """
    while True:
        valor_input = input(texto_input)
        if valor_input.isdigit() and int(valor_input) > 0:
            return int(valor_input)
        print(">> Valor inválido! Digite um número inteiro positivo.")


def mostra_menu(titulo, opcoes):
    """
    Exibe um menu com opções para o usuário.

    :param titulo: Título do menu
    :param opcoes: Lista de opções do menu
    """
    max_length = max(len(option) for option in opcoes)
    max_length = len(titulo) if len(titulo) >= max_length else max_length
    border_length = max_length + 6
    print("\n╔" + "═" * border_length + "╗")
    print(f"║ {titulo.center(border_length - 2)} ║")
    print("╚" + "═" * border_length + "╝")
    for i, option in enumerate(opcoes, start=1):
        print(f"║ {i} - {option.ljust(max_length)} ║")
    print("╚" + "═" * border_length + "╝")
    print("║ S - Sair ".ljust(max_length + 7) + "║")
    print("╚" + "═" * border_length + "╝")


def seleciona_escolha(titulo, opcoes):
    """
    Garante que o usuario escolha entre o vetor de opções informado.

    :param titulo: Título do menu
    :param opcoes: Lista de opções do menu
    :return: Indice da opção
    """
    while True:
        mostra_menu(titulo, opcoes)
        choice = input("\n>> ").strip().upper()
        if choice == "S":
            return "S"
        try:
            choice = int(choice)
            if 1 <= choice <= len(opcoes):
                return choice
            else:
                print(">> Opção inválida!")
        except ValueError:
            print(">> Opção inválida!")

def gerar_excel(nome_arquivo):
    """
    Gera um excel com os dados gerados nessa aplicação.

    :param nome_arquivo: Nome do arquivo a ser gerado.
    """
    dados_ruas = []
    dados_insumos = []
    dados_talhoes = {
        'retangulo': [],
        'triangulo': [],
        'trapezio': []
    }

    for cultura in culturas:
        for i, talhao in enumerate(cultura.talhoes):
            nome = talhao['nome']
            forma = talhao['forma']
            data = talhao['data']

            if forma == 'retangulo':
                comprimento = talhao['comprimento']
                largura = talhao['largura']
                area_total = cultura.calcular_area_retangulo(i, comprimento, largura)
                dados_talhoes['retangulo'].append([cultura.nome, nome, forma, comprimento, largura, area_total, data])

            if forma == 'triangulo':
                base = talhao['base']
                altura = talhao['altura']
                area_total = cultura.calcular_area_triangulo(i, base, altura)
                dados_talhoes['triangulo'].append([cultura.nome, nome, forma, base, altura, area_total, data])

            if forma == 'trapezio':
                base_maior = talhao['base_maior']
                base_menor = talhao['base_menor']
                altura = talhao['altura']
                area_total = cultura.calcular_area_trapezio(i, base_maior, base_menor, altura)
                dados_talhoes['trapezio'].append([cultura.nome, nome, forma, base_maior, base_menor, altura, area_total, data])

            for rua in talhao["ruas"]:
                nome = rua['nome']
                comprimento = rua['comprimento']
                largura = rua['largura']
                data = rua['data']
                area = comprimento * largura
                dados_ruas.append([cultura.nome, talhao["nome"], nome, comprimento, largura, area, data])

            for produto in cultura.produtos:
                nome = produto['nome']
                quantidade = produto['quantidade']
                data = produto['data']
                total_insumo = area_total * (quantidade / 1000)
                dados_insumos.append([cultura.nome, talhao["nome"], nome, quantidade, total_insumo, data])



    df_dados_ruas = pd.DataFrame(dados_ruas, columns=["Cultura", "Talhao", "Nome", "Comprimento", "Largura", "Área Total", "Data"])
    df_dados_insumos = pd.DataFrame(dados_insumos, columns=["Cultura", "Talhao", "Nome", "Quantidade", "Total Necessário", "Data"])

    # Simulando os DataFrames de exemplo
    df_talhoes_retangulo = pd.DataFrame(
        dados_talhoes['retangulo'],
        columns=["Cultura", "Nome", "Formato", "Comprimento", "Largura", "Área Total", "Data"]
    )

    df_talhoes_triangulo = pd.DataFrame(
        dados_talhoes['triangulo'],
        columns=["Cultura", "Nome", "Formato", "Base", "Altura", "Área Total", "Data"]
    )

    df_talhoes_trapezio = pd.DataFrame(
        dados_talhoes['trapezio'],
        columns=["Cultura", "Nome", "Formato", "Base Maior", "Base Menor", "Altura", "Área Total", "Data"]
    )

    # Fazendo o merge (concatenação) dos DataFrames
    df_dados_talhoes = pd.concat(
        [df_talhoes_retangulo, df_talhoes_triangulo, df_talhoes_trapezio],
        axis=0,  # Concatenar ao longo das linhas (verticalmente)
        ignore_index=True,  # Ignorar os índices originais e gerar novos
        sort=False  # Manter a ordem das colunas sem ordenação
    )

    # Salvando em um arquivo Excel com duas abas
    with pd.ExcelWriter(nome_arquivo) as writer:
        df_dados_ruas.to_excel(writer, sheet_name="Dados de ruas", index=False)
        df_dados_insumos.to_excel(writer, sheet_name="Dados de Insumos", index=False)
        df_dados_talhoes.to_excel(writer, sheet_name="Dados de talhões", index=False)

## Definição da Classe CulturaAgricola

A classe `CulturaAgricola` é responsável por armazenar informações sobre uma cultura agrícola específica, incluindo os talhões, as ruas da plantação, a forma geométrica usada para calcular a área de plantio e a quantidade de insumo por metro. Também inclui métodos para calcular a área de plantio e o manejo de insumos.


In [18]:
class CulturaAgricola:
    def __init__(self, nome, produtos, talhoes):
        """
        Inicializa a cultura agrícola.

        :param nome: Nome da cultura (e.g., "Café", "Cana-de-Açúcar")
        :param produtos: Lista de dicionário de produtos e suas respectivas doses
        :param talhões: Lista de dicionário de talhões de plantio
        """
        self.nome = nome
        self.talhoes = talhoes
        self.produtos = produtos

    def lista_ruas(self, indice_talhao):
        """
        Lista as ruas cadastradas

        :param indice_talhao: Indice do talhão (e.g 0, 1)
        """

        for rua in self.talhoes[indice_talhao]["ruas"]:
            nome = rua["nome"]
            comprimento = rua["comprimento"]
            largura = rua["largura"]
            total = comprimento * largura
            print(f">> Nome: {nome} Comprimento: {comprimento} Largura: {largura} Total: {total}m²")

    def adicionar_rua(self, indice_talhao):
        """
        Adiciona uma rua na lista de ruas do talhão.
        :param indice_talhao: Indice do talhão (e.g 0, 1)
        """
        nome = input(">> Digite o nome da rua")
        comprimento = input_numerico(">> Digite o comprimento da rua")
        largura = input_numerico(">> Digite a largura da rua")
        data = datetime.now().strftime('%d-%m-%y')

        self.talhoes[indice_talhao]["ruas"].append({"nome": nome, "comprimento": comprimento, "largura": largura, "data": data})

    def editar_rua(self, indice_talhao):
        """
        Edita os valores de uma rua do talhão

        :param indice_talhao: Indice do talhão (e.g 0, 1)
        """
        if len(self.talhoes[indice_talhao]["ruas"]) == 0:
            print(">> Lista de ruas esta vazio")
            return

        indice = seleciona_escolha("Editar rua", [str(r["nome"]) for r in self.talhoes[indice_talhao]["ruas"]])

        if indice != "S":
            nome = input(">> Digite o nome da rua")
            comprimento = input_numerico(">> Digite o comprimento da rua")
            largura = input_numerico(">> Digite a largura da rua")
            data = datetime.now()

            self.talhoes[indice_talhao]["ruas"][indice -1] = {"nome": nome, "comprimento": comprimento, "largura": largura, "data": data}

    def remover_rua(self, indice_talhao):
        """
        Remove item da lista de ruas do talhão.

        :param indice_talhao: Indice do talhão (e.g 0, 1)
        """
        if len(self.talhoes[indice_talhao]["ruas"]) == 0:
            print(">> Lista de ruas esta vazio")
            return

        indice = seleciona_escolha("Remover rua", [str(r["nome"]) for r in self.talhoes[indice_talhao]["ruas"]])

        if indice != "S":
            del self.talhoes[indice_talhao]["ruas"][indice -1]

    def lista_produtos(self):
        """
        Lista os produtos cadastradas
        """
        for produto in self.produtos:
            nome = produto["nome"]
            quantidade = produto["quantidade"]
            data = produto["data"]

            print(f">> Nome: {nome} Quantidade: {quantidade}mL")

    def adicionar_produto(self):
        """
        Adiciona um produto ao cálculo de manejo de insumos com data de aplicação.
        """
        nome = input(">> Digite o nome do produto")
        quantidade = input_numerico(">> Digite digite a quantidade de insumo(mL) por metro")

        self.produtos.append({
            'nome': nome,
            'quantidade': quantidade,
            'data': datetime.now()
        })

    def editar_produto(self):
        """
        Edita um produto do manejo de insumos.
        """
        if len(self.produtos) == 0:
            print(">> Lista de produtos esta vazia")
            return

        indice = seleciona_escolha(">> Editar produto", [str(p["nome"]) for p in self.produtos])

        if indice != "S":
            nome = input(">> Digite o nome do produto")
            quantidade = input_numerico(">> Digite digite a quantidade de insumo(mL) por metro")
            self.produtos[indice -1] = {
                'nome': nome,
                'quantidade': quantidade,
                'data': datetime.now()
            }

    def remover_produto(self):
        """
        Remove um produto da cultura agrícola.
        """
        if len(self.produtos) == 0:
            print(">> Lista de produtos esta vazia")
            return

        indice = seleciona_escolha(">> Remover produto", [str(p["nome"]) for p in self.produtos])

        if indice != "S":
            del self.produtos[indice -1]

    def lista_talhoes(self):
        """
        Lista os talhões cadastradas
        """
        for i, talhao in enumerate(self.talhoes):
            nome = talhao["nome"]
            forma = talhao["forma"]
            data = talhao["data"]

            if forma == 'retangulo':
                comprimento = talhao["comprimento"]
                largura =  talhao["largura"]
                area = self.calcular_area_retangulo(i, comprimento, largura)
                print(f">> Nome: {nome} Forma: Retângulo Comprimento: {comprimento} Largura: {largura} Área plantio: {area}m²")

            if forma == 'triangulo':
                base = talhao["base"]
                altura =  talhao["altura"]
                area = self.calcular_area_triangulo(i, base, altura)
                print(f">> Nome: {nome} Forma: Triângulo Base: {base} Altura: {altura} Área plantio: {area}m²")

            if forma == 'trapezio':
                base_maior = talhao["base_maior"]
                base_menor =  talhao["base_menor"]
                altura =  talhao["altura"]
                area = self.calcular_area_trapezio(i, base_maior, base_menor, altura)
                print(f">> Nome: {nome} Forma: Trapézio Base maior: {base_maior} Base menor: {base_menor} Altura: {altura} Área plantio: {area}m²")

    def adicionar_talhao(self):
        """
        Adiciona um produto ao cálculo de manejo de insumos com data de aplicação.
        """

        indice = seleciona_escolha(">> Selecionar formato do talhão", ["Retângulo", "Triângulo", "Trapézio"])

        nome = input(">> Digite o nome do talhão")
        if indice == 1:
            comprimento = input_numerico(">> Digite o comprimento do talhão")
            largura = input_numerico(">> Digite o largura do talhão")

            self.talhoes.append({
                "nome": nome,
                "forma": "retangulo",
                "comprimento": comprimento,
                "largura": largura,
                "data": datetime.now(),
                "ruas": []
            })

        if indice == 2:
            base = input_numerico(">> Digite a base do talhão")
            altura = input_numerico(">> Digite a altura do talhão")

            self.talhoes.append({
                "nome": nome,
                "forma": "triangulo",
                "base": base,
                "altura": altura,
                "data": datetime.now(),
                "ruas": []
            })

        if indice == 3:
            base_maior = input_numerico(">> Digite a base maior do talhão")
            base_menor = input_numerico(">> Digite a base menor do talhão")
            altura = input_numerico(">> Digite a altura do talhão")

            self.talhoes.append({
                "nome": nome,
                "forma": "trapezio",
                "base_maior": base_maior,
                "base_menor": base_menor,
                "altura": altura,
                "data": datetime.now(),
                "ruas": []
            })

    def editar_talhao(self):
        """
        Edita um produto do manejo de insumos.
        """
        if len(self.talhoes) == 0:
            print(">> Lista de talhões esta vazia")
            return

        indice = seleciona_escolha(">> Editar talhão", [str(t["nome"]) for t in self.talhoes])

        if indice != "S":
            indice_forma = seleciona_escolha("Selecionar formato do talhão", ["Retângulo", "Triângulo", "Trapézio"])

            nome = input(">> Digite o nome do talhão")
            ruas = self.talhoes[indice -1]["ruas"]

            if indice_forma == 1:
                comprimento = input_numerico(">> Digite o comprimento do talhão")
                largura = input_numerico(">> Digite o largura do talhão")

                self.talhoes[indice -1] = {
                    "nome": nome,
                    "forma": "retangulo",
                    "comprimento": comprimento,
                    "largura": largura,
                    "data": datetime.now(),
                    "ruas": ruas
                }

            if indice_forma == 2:
                base = input_numerico(">> Digite a base do talhão")
                altura = input_numerico(">> Digite a altura do talhão")

                self.talhoes[indice -1] = {
                    'nome': nome,
                    'forma': "triangulo",
                    'base': base,
                    'altura': altura,
                    'data': datetime.now(),
                    "ruas": ruas
                }

            if indice_forma == 3:
                base_maior = input_numerico(">> Digite a base maior do talhão")
                base_menor = input_numerico(">> Digite a base menor do talhão")
                altura = input_numerico(">> Digite a altura do talhão")

                self.talhoes[indice -1] = {
                    "nome": nome,
                    "forma": "trapezio",
                    "base_maior": base_maior,
                    "base_menor": base_menor,
                    "altura": altura,
                    "data": datetime.now(),
                    "ruas": ruas
                }

    def remover_talhao(self):
        """
        Remove um talhao da cultura agrícola.
        """
        if len(self.talhoes) == 0:
            print(">> Lista de talhões esta vazia")
            return

        indice = seleciona_escolha(">> Remover talhão", [str(t["nome"]) for t in self.talhoes])

        if indice != "S":
            del self.talhoes[indice -1]

    def calcular_area_retangulo(self, indice_talhao, comprimento, largura):
        """
        Calcula a área de plantio na forma retangular. (altura * largura) - ruas
        :param indice_talhao: Indice do talhão para ser utilizado no calculo (e.g: 0, 1)
        :param comprimento: Comprimento do talhão em metros
        :param largura: Largura do talhão em metros
        :return: Área calculada
        """
        area_retangulo = (comprimento * largura)
        metragem_ruas = sum((r["comprimento"] * r["largura"]) for r in self.talhoes[indice_talhao]["ruas"])

        area_util_plantio = area_retangulo - metragem_ruas

        return area_util_plantio

    def calcular_area_triangulo(self, indice_talhao, base, altura):
        """
        Calcula a área de plantio na forma triangular. ((base * altura) / 2) - Ruas
        :param indice_talhao: Indice do talhão para ser utilizado no calculo (e.g: 0, 1)
        :param base: Base do talhão em metros
        :param altura: Altura do talhão em metros
        :return: Área calculada
        """
        area_triangulo = (base * altura) / 2
        metragem_ruas = sum((r["comprimento"] * r["largura"]) for r in self.talhoes[indice_talhao]["ruas"])

        area_util_plantio = area_triangulo - metragem_ruas

        return area_util_plantio

    def calcular_area_trapezio(self, indice_talhao, base_maior, base_menor, altura):
        """
        Calcula a área de plantio na forma de trapézio. (((base_maior + base_menor) * altura) / 2) - ruas
        :param indice_talhao: Indice do talhão para ser utilizado no calculo (e.g: 0, 1)
        :param base_maior: Base maior do talhão em metros
        :param base_menor: Base menor do talhão em metros
        :param altura: Altura do talhão em metros
        :return: Área calculada
        """
        area_trapezio = ((base_maior + base_menor) * altura) / 2
        metragem_ruas = sum((r["comprimento"] * r["largura"]) for r in self.talhoes[indice_talhao]["ruas"])

        area_util_plantio = area_trapezio - metragem_ruas

        return area_util_plantio

    def calcular_manejo_insumos(self, indice_talhao):
        """
        Calcula a quantidade total de insumos necessária para cada produto.
        :param indice_talhao: Indice do talhão para ser utilizado no calculo (e.g: 0, 1)
        """
        talhao = self.talhoes[indice_talhao]
        area_plantio = None

        for produto in self.produtos:
            nome = produto["nome"]
            quantidade = produto["quantidade"]

            if talhao["forma"] == "retangulo":
                area_plantio = self.calcular_area_retangulo(indice_talhao, talhao["comprimento"], talhao["largura"])

            if talhao["forma"] == "triangulo":
                area_plantio = self.calcular_area_triangulo(indice_talhao, talhao["base"], talhao["altura"])

            if talhao["forma"] == "trapezio":
                area_plantio = self.calcular_area_trapezio(indice_talhao, talhao["base_maior"], talhao["base_menor"], talhao["altura"])

            total_insumo = area_plantio * (quantidade / 1000)
            print(f">> Nome: {nome} Quantidade: {quantidade}mL Total necessario: {round(total_insumo, 2)}L")

    def exibir_informacoes(self):
        """
        Exibe as informações da cultura agrícola.
        """
        print("=============================================================")
        print(f">> Cultura: {self.nome}")
        for i, talhao in enumerate(self.talhoes):
            nome = talhao["nome"]
            forma = talhao["forma"]
            ruas = talhao["ruas"]
            data = talhao["data"]

            if forma == 'retangulo':
                comprimento = talhao["comprimento"]
                largura =  talhao["largura"]
                area = self.calcular_area_retangulo(i, comprimento, largura)
                print(f">> Nome: {nome} Forma: Retângulo Comprimento: {comprimento} Largura: {largura} Área plantio: {area}m²")

            if forma == 'triangulo':
                base = talhao["base"]
                altura =  talhao["altura"]
                area = self.calcular_area_triangulo(i, base, altura)
                print(f">> Nome: {nome} Forma: Triângulo Base: {base} Altura: {altura} Área plantio: {area}m²")

            if forma == 'trapezio':
                base_maior = talhao["base_maior"]
                base_menor =  talhao["base_menor"]
                altura =  talhao["altura"]
                area = self.calcular_area_trapezio(i, base_maior, base_menor, altura)
                print(f">> Nome: {nome} Forma: Trapézio Base maior: {base_maior} Base menor: {base_menor} Altura: {altura} Área plantio: {area}m²")

            print(">> Ruas")
            self.lista_ruas(i)
            print(">> Insumos")
            self.calcular_manejo_insumos(i)
            print("=============================================================")



## Definição de Valores Iniciais

Nesta seção, definimos os valores iniciais necessários para a execução do sistema. Esses valores incluem a criação das instâncias das culturas agrícolas, o mapeamento de produtos aos seus respectivos insumos por metro e a configuração das ruas para cada talhão.

**Importante:** Os valores definidos aqui são apenas valores iniciais e são **opcionais**. Você pode alterar esses valores ao longo do uso do sistema, conforme necessário. Isso permite ajustar as configurações para refletir diferentes cenários ou requisitos específicos da sua análise agrícola.

Esses valores são usados para realizar os cálculos de área de plantio e manejo de insumos. Se você quiser modificar qualquer configuração, basta alterar os valores na seção apropriada do código.


### Nota Importante

O cálculo de área plantada assume que as medidas fornecidas estão em metros. Certifique-se de converter outras unidades antes de inseri-las.


In [19]:
# Valores iniciais dos talhões
talhoes = {
    "soja": [
        {
            "nome": "Talhão de soja 1",
            "forma": "retangulo",
            "comprimento": 350,
            "largura": 150,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 150, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 180, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 160, "largura": 3, "data": datetime(2024, 9, 13)}
            ]
        },
        {
            "nome": "Talhão de soja 2",
            "forma": "retangulo",
            "comprimento": 400,
            "largura": 180,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 200, "largura": 3.5, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 210, "largura": 3.5, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 180, "largura": 3.5, "data": datetime(2024, 9, 13)}
            ]
        },
        {
            "nome": "Talhão de soja 3",
            "forma": "triangulo",
            "base": 380,
            "altura": 170,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 150, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 140, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 160, "largura": 3, "data": datetime(2024, 9, 13)}
            ]
        },
        {
            "nome": "Talhão de soja 4",
            "forma": "triangulo",
            "base": 420,
            "altura": 190,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 200, "largura": 3.5, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 220, "largura": 3.5, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 190, "largura": 3.5, "data": datetime(2024, 9, 13)}
            ]
        },
    ],
    "cafe": [
        {
            "nome": "Talhão de cafe 1",
            "forma": "retangulo",
            "comprimento": 100,
            "largura": 50,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 90, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 85, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 95, "largura": 3, "data": datetime(2024, 9, 13)}
            ]
        },
        {
            "nome": "Talhão de cafe 2",
            "forma": "retangulo",
            "comprimento": 120,
            "largura": 60,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 110, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 115, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 120, "largura": 3, "data": datetime(2024, 9, 13)}
            ]
        },
        {
            "nome": "Talhão de cafe 3",
            "forma": "retangulo",
            "comprimento": 110,
            "largura": 55,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 105, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 100, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 107, "largura": 3, "data": datetime(2024, 9, 13)}
            ]
        },
        {
            "nome": "Talhão de cafe 4",
            "forma": "retangulo",
            "comprimento": 130,
            "largura": 65,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 125, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 120, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 130, "largura": 3, "data": datetime(2024, 9, 13)}
            ]
        }
    ],
    "milho": [
        {
            "nome": "Talhão de milho 1",
            "forma": "retangulo",
            "comprimento": 300,
            "largura": 120,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 280, "largura": 4, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 290, "largura": 4, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 300, "largura": 4, "data": datetime(2024, 9, 13)}
            ]
        },
        {
            "nome": "Talhão de milho 2",
            "forma": "retangulo",
            "comprimento": 320,
            "largura": 140,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 310, "largura": 4, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 315, "largura": 4, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 320, "largura": 4, "data": datetime(2024, 9, 13)}
            ]
        },
        {
            "nome": "Talhão de milho 3",
            "forma": "trapezio",
            "base_maior": 330,
            "base_menor": 150,
            "altura": 130,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 320, "largura": 4, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 310, "largura": 4, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 330, "largura": 4, "data": datetime(2024, 9, 13)}
            ]
        },
        {
            "nome": "Talhão de milho 4",
            "forma": "trapezio",
            "base_maior": 350,
            "base_menor": 170,
            "altura": 150,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 340, "largura": 4, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 330, "largura": 4, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 350, "largura": 4, "data": datetime(2024, 9, 13)}
            ]
        }
    ],
    "feijao": [
        {
            "nome": "Talhão de feijão 1",
            "forma": "triangulo",
            "base": 220,
            "altura": 100,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 180, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 190, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 200, "largura": 3, "data": datetime(2024, 9, 13)}
            ]
        },
        {
            "nome": "Talhão de feijão 2",
            "forma": "triangulo",
            "base": 240,
            "altura": 110,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 200, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 210, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 220, "largura": 3, "data": datetime(2024, 9, 13)}
            ]
        },
        {
            "nome": "Talhão de feijão 3",
            "forma": "trapezio",
            "base_maior": 250,
            "base_menor": 115,
            "altura": 105,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 230, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 240, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 250, "largura": 3, "data": datetime(2024, 9, 13)}
            ]
        },
        {
            "nome": "Talhão de feijão 4",
            "forma": "trapezio",
            "base_maior": 265,
            "base_menor": 130,
            "altura": 115,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 250, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 255, "largura": 3, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 265, "largura": 3, "data": datetime(2024, 9, 13)}
            ]
        }
    ],
    "cana": [
        {
            "nome": "Talhão de cana-de-açúcar 1",
            "forma": "triangulo",
            "base": 500,
            "altura": 250,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 300, "largura": 4, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 320, "largura": 4, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 350, "largura": 4, "data": datetime(2024, 9, 13)}
            ]
        },
        {
            "nome": "Talhão de cana-de-açúcar 2",
            "forma": "triangulo",
            "base": 520,
            "altura": 260,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 310, "largura": 4, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 330, "largura": 4, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 340, "largura": 4, "data": datetime(2024, 9, 13)}
            ]
        },
        {
            "nome": "Talhão de cana-de-açúcar 3",
            "forma": "trapezio",
            "base_maior": 530,
            "base_menor": 270,
            "altura": 265,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 350, "largura": 4, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 360, "largura": 4, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 370, "largura": 4, "data": datetime(2024, 9, 13)}
            ]
        },
        {
            "nome": "Talhão de cana-de-açúcar 4",
            "forma": "trapezio",
            "base_maior": 550,
            "base_menor": 290,
            "altura": 275,
            "data": datetime(2024, 9, 13),
            "ruas": [
                {"nome": "Rua 1", "comprimento": 360, "largura": 4, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 2", "comprimento": 370, "largura": 4, "data": datetime(2024, 9, 13)},
                {"nome": "Rua 3", "comprimento": 380, "largura": 4, "data": datetime(2024, 9, 13)}
            ]
        }
    ],
}

# Produtos utilizados com data de aplicação
produtos = {
    'soja': [
        {'nome': 'Herbicida', 'quantidade': 50, 'data': datetime(2024, 9, 1)},
        {'nome': 'Fertilizante', 'quantidade': 30, 'data': datetime(2024, 9, 5)},
        {'nome': 'Inoculante', 'quantidade': 20, 'data': datetime(2024, 9, 10)}
    ],
    'cafe': [
        {'nome': 'Fosfato', 'quantidade': 40, 'data': datetime(2024, 9, 2)},
        {'nome': 'Adubo', 'quantidade': 30, 'data': datetime(2024, 9, 6)},
        {'nome': 'Calda Bordalesa', 'quantidade': 60, 'data': datetime(2024, 9, 12)}
    ],
    'milho': [
        {'nome': 'Herbicida', 'quantidade': 45, 'data': datetime(2024, 9, 3)},
        {'nome': 'Fertilizante', 'quantidade': 25, 'data': datetime(2024, 9, 8)},
        {'nome': 'Inseticida', 'quantidade': 15, 'data': datetime(2024, 9, 15)}
    ],
    'feijao': [
        {'nome': 'Fertilizante', 'quantidade': 20, 'data': datetime(2024, 9, 4)},
        {'nome': 'Herbicida', 'quantidade': 40, 'data': datetime(2024, 9, 9)},
        {'nome': 'Inoculante', 'quantidade': 10, 'data': datetime(2024, 9, 14)}
    ],
    'cana': [
        {'nome': 'Adubo', 'quantidade': 50, 'data': datetime(2024, 9, 7)},
        {'nome': 'Herbicida', 'quantidade': 70, 'data': datetime(2024, 9, 11)},
        {'nome': 'Fertilizante', 'quantidade': 30, 'data': datetime(2024, 9, 16)}
    ]
}

# Vetor da classe de Culturas agricolas
culturas = [
    CulturaAgricola(nome="Soja", talhoes=talhoes['soja'], produtos=produtos['soja']),
    CulturaAgricola(nome="Café", talhoes=talhoes['cafe'], produtos=produtos['cafe']),
    CulturaAgricola(nome="Milho", talhoes=talhoes['milho'], produtos=produtos['milho']),
    CulturaAgricola(nome="Feijão", talhoes=talhoes['feijao'], produtos=produtos['feijao']),
    CulturaAgricola(nome="Cana-de-Açúcar", talhoes=talhoes['cana'], produtos=produtos['cana']),
]

## Como Usar Este Notebook

1. **Selecione a cultura**: Escolha a cultura agrícola para a qual você deseja calcular a área de plantio ou o manejo de insumos.
2. **Talhões**: Insira os dados do talhão para calcular sua area de plantio.
3. **Ruas**: Insira o comprimento e largura das ruas para o talhão selecionado.
4. **Defina os produtos**: Defina o produto e a quantidade de insumo necessária por metro do talhão.
5. **Exportar os dados**: Exporte um excel com todas as informações geradas nesse sistema para insumo da sua cultura agrícola.
6. **Dados meteorológicos**: Consulte a api em tempo real de WeatherForecast, para insumos relacionados ao clima.


In [ ]:
def main():
    cultura_atual = None
    indice_talhao = None
    acao_rua = False
    acao_insumo = False
    acao_talhoes = False
    acao_dados_meterologicos = False
    sair = False

    while not sair:
        # Selecionar a cultura
        if not cultura_atual:
            clear_output(wait=True)

            opcoes = [c.nome for c in culturas]
            indice = seleciona_escolha("Culturas", opcoes)

            if indice == "S":
                sair = True

            else:
                cultura_atual = culturas[indice - 1]

        if cultura_atual:
            clear_output(wait=True)

            if not acao_rua and not acao_insumo and not acao_talhoes and not acao_dados_meterologicos:
                opcoes = ["Talhões", "Insumos", "Exibir informações", "Exportar Dados", "Consultar Dados Meteorológicos", "Voltar"]
                input_acao = seleciona_escolha(f"{cultura_atual.nome} > Ações", opcoes)

                if input_acao == 1:
                    acao_talhoes = True

                elif input_acao == 2:
                    acao_insumo = True

                elif input_acao == 3:
                    cultura_atual.exibir_informacoes()
                    input(">> Pressione uma tecla para continuar...")

                elif input_acao == 4:
                    nome_arquivo = input(">> Digite o nome de saida do arquivo: ")
                    gerar_excel(nome_arquivo=f"{nome_arquivo}.xlsx")

                elif input_acao == 5:
                    acao_dados_meterologicos = True

                elif input_acao == 6:
                    cultura_atual = False

                elif input_acao == "S":
                    sair = True

            if acao_rua and acao_talhoes:
                clear_output(wait=True)

                talhao_atual = cultura_atual.talhoes[indice_talhao]
                opcoes = ["Adicionar", "Editar", "Listar", "Remover", "Voltar"]
                input_rua = seleciona_escolha(f"{cultura_atual.nome} > Ações > {talhao_atual['nome']} > Ruas", opcoes)

                if input_rua == 1:
                    cultura_atual.adicionar_rua(indice_talhao)

                elif input_rua == 2:
                    cultura_atual.editar_rua(indice_talhao)

                elif input_rua == 3:
                    cultura_atual.lista_ruas(indice_talhao)
                    input(">> Pressione uma tecla para continuar...")

                elif input_rua == 4:
                    cultura_atual.remover_rua(indice_talhao)

                elif input_rua == 5:
                    acao_rua = False

                elif input_rua == "S":
                    sair = True

            if acao_insumo:
                clear_output(wait=True)

                opcoes = ["Adicionar", "Editar", "Listar", "Remover", "Voltar"]
                input_insumo = seleciona_escolha(f"{cultura_atual.nome} > Ações > Insumos", opcoes)

                if input_insumo == 1:
                    cultura_atual.adicionar_produto()

                elif input_insumo == 2:
                    cultura_atual.editar_produto()

                elif input_insumo == 3:
                    cultura_atual.lista_produtos()
                    input(">> Pressione uma tecla para continuar...")

                elif input_insumo == 4:
                    cultura_atual.remover_produto()

                elif input_insumo == 5:
                    acao_insumo = False

                elif input_insumo == "S":
                    sair = True

            if acao_talhoes and not acao_rua:
                clear_output(wait=True)

                opcoes = ["Adicionar", "Editar", "Listar", "Remover", "Ruas", "Voltar"]
                input_talhao = seleciona_escolha(f"{cultura_atual.nome} > Ações > Talhões", opcoes)

                if input_talhao == 1:
                    cultura_atual.adicionar_talhao()

                elif input_talhao == 2:
                    cultura_atual.editar_talhao()

                elif input_talhao == 3:
                    cultura_atual.lista_talhoes()
                    input(">> Pressione uma tecla para continuar...")

                elif input_talhao == 4:
                    cultura_atual.remover_talhao()

                elif input_talhao == 5:
                    indice = seleciona_escolha("Selecionar talhão", [str(t["nome"]) for t in cultura_atual.talhoes])

                    if indice != "S":
                        acao_rua = True
                        indice_talhao = indice -1

                elif input_talhao == 6:
                    acao_talhoes = False

                elif input_talhao == "S":
                    sair = True

            if acao_dados_meterologicos:
                clear_output(wait=True)

                # Criar um fluxo vazio
                empty_stream = io.StringIO()

                # Substituir os callbacks padrões para redirecionar a saída
                rpy2.rinterface_lib.callbacks.consolewrite_print = lambda x: empty_stream.write(x)
                rpy2.rinterface_lib.callbacks.consolewrite_warnerror = lambda x: empty_stream.write(x)

                # URL do script R
                url_r = "https://raw.githubusercontent.com/Yuri-Cordeiro/FarmTechSolutions/refs/heads/main/dados_meteorologicos.r"

                # Fazer a requisição para obter o conteúdo do arquivo
                response = requests.get(url_r)

                # Verificar se a requisição foi bem-sucedida
                if response.status_code == 200:
                    # Ler o conteúdo do script como texto
                    script_r = response.text
                else:
                    print(f"Erro ao acessar o arquivo: {response.status_code}")

                # Executar o script R
                r(script_r)

                # Chamar a função acao_dados_metereologicos do R e capturar o retorno
                acao_dados_metereologicos_func = r['acao_dados_metereologicos']

                resultados = acao_dados_metereologicos_func()


                opcoes = [
                    "Probabilidade média de chuva",
                    "Temperatura média",
                    "Velocidade média de vento",
                    "Volume médio de umidade",
                    "Voltar",
                ]
                input_meteorologico = seleciona_escolha("Dados meteorológicos", opcoes)

                if input_meteorologico == 1:
                    avg_precipitacao = resultados.rx2('avg_precipitacao')[0]  # Probabilidade média de precipitação
                    print(f"A previsao da probabilidade media de chuva é:{avg_precipitacao}%")
                    input(">> Pressione uma tecla para continuar...")

                elif input_meteorologico == 2:
                    avg_temperatura = resultados.rx2('avg_temperatura')[0]  # Temperatura média
                    print(f'A previsão de temperatura media para os proximos dias é:{avg_temperatura}°C')
                    input(">> Pressione uma tecla para continuar...")

                elif input_meteorologico == 3:
                    avg_vento = resultados.rx2('avg_vento')[0]  # Velocidade média do vento
                    print(f'A previsao da velocidade media do vento para os proximos dias é:{avg_vento}m/s')
                    input(">> Pressione uma tecla para continuar...")

                elif input_meteorologico == 4:
                    avg_umidade = resultados.rx2('avg_umidade')[0]  # Umidade média
                    print(f'A previsao  de umidade media para os proximos dias é:{avg_umidade}%')
                    input(">> Pressione uma tecla para continuar...")

                elif input_meteorologico == 5:
                    acao_dados_meterologicos = False

                elif input_meteorologico == "S":
                    sair = True

if __name__ == '__main__':
    main()